In [2]:
import re
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import string
import pandas as pd
import mtranslate
import re

In [3]:
RE_EMOJI = re.compile('[^\U00000000-\U0000d7ff\U0000e000-\U0000ffff]', flags=re.UNICODE)

def strip_emoji(text):
    return RE_EMOJI.sub(r'', text)

In [4]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [5]:
puncts='.?!,]@'
df = pd.DataFrame(columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()
# words_to_remove = ['training', 'all', 'wow', 'fun']
# words_to_detect = ['promo', 'promotion', 'birthday', 'yoga']
#For some reason, please returns positive sentiment because it assumes 'pleasing' but normally people just asking politely
weird_sentiment_words = ['i', 'la', 'please', 'lah', 'dm']

file = json.load(open('instagram_chifitness6months.json', encoding="utf8", errors="ignore"))
for line in file['data']:
    try:
        print("New comment")
        print('---------------------------------------')
        sentence = line['comment']

        #Translate to English
        sentence = mtranslate.translate(sentence, 'en', 'auto')

        #Split a single comment into multiple sentences
        sentences = sent_tokenize(sentence)
        #print(sentences)

        #Sentiment for each sentence
        for s in sentences:

            s = s.lower()
            s = s.replace('chifitness_malaysia', ' ')
            s = s.replace('chi_fitness_malaysia', ' ')
            s = s.replace('chi fitness', ' ')
            s = s.replace('chi', ' ')
            s = s.replace('chi-fitness', ' ')
            s = s.replace('chi_fitness', ' ')
            s = re.sub(r"\byin\b", "", s)
            s = re.sub(r"\byang\b", "", s)

            print("Sentence: ", s)
            sentiment = 0.0
            #print(s)

            tokenized = word_tokenize(s)
            #print(tokenized)

            new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
            tokenized = new_tokenized

            #POS tag each word
            tagged = pos_tag(tokenized)
            print(tagged)

            previous_word = ""
            list_of_nouns = []

            #Get the sentiment for each word
            for w, t in tagged:


                sentiment_added = False

                #Convert Penn Treebank POS tag to Wordnet POS tag
                wn_tag = penn_to_wn(t)

                #If successfully converted, lemmatize using the Wordnet POS tag
                #If not successfully converted, lemmatize without any POS tags
                if wn_tag is None:
                    lemma = lemmatizer.lemmatize(w)
                else:
                    lemma = lemmatizer.lemmatize(w, pos=wn_tag)

                #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                if not lemma:
                    print("Can't lemmatize: ", w)
                    final_word = w
                else:
                    print("Lemmatized word: ", lemma)
                    final_word = lemma

                #Make cool and wow positive words
                if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                    print('Not is the previous word')
                    sentiment = sentiment - 0.5
                    previous_word = final_word
                    sentiment_added = True
                elif (final_word in ['cool', 'wow', 'glad', 'lose', 'lost']):
                    print(final_word, " has a score of 0.5")
                    sentiment = sentiment + 0.5
                    previous_word = final_word
                    sentiment_added = True
                elif (final_word == 'nice') and (previous_word == 'not'):
                    print('Not is the previous word')
                    sentiment = sentiment - 0.875
                    previous_word = final_word
                    sentiment_added = True
                elif (final_word == 'nice'):
                    print(final_word, " has a score of 0.875")
                    sentiment = sentiment + 0.875
                    previous_word = final_word
                    sentiment_added = True
                elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                    sentiment = sentiment + 0.75
                    print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                    previous_word = final_word
                    sentiment_added = True
                elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                    sentiment = sentiment - 0.75
                    print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                    previous_word = final_word
                    sentiment_added = True

                #If the Wordnet POS tag exists, use it to get the synset of the word
                if wn_tag is None:
                    synsets = wn.synsets(final_word)
                else:
                    synsets = wn.synsets(final_word, pos=wn_tag)

                #If no synsets found, skip the word
                if not synsets:
                    print("Synset not found")
                    continue

                # Take the first sense of the word which is the most common
                synset = synsets[0]
                swn_synset = swn.senti_synset(synset.name())

                if sentiment_added == False:
                    if final_word != 'fitness' and previous_word == 'not':      
                        print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                        sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                    elif final_word != 'fitness':
                        print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                        sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                    else:
                        print("The word fitness is not considered as a sentiment")

                previous_word = final_word

                #Extract nouns
                if t.startswith('N') or w == 'post':
                    if w != 'i' and w != 'la' and w != 'hi':
                        list_of_nouns.append(w)

            try:
                df2 = pd.DataFrame([[line['post'], s, ','.join(list_of_nouns), sentiment, 'Instagram Comment Chi Fitness']],columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])
            except Exception:
                df2 = pd.DataFrame([['No message', s, ','.join(list_of_nouns), sentiment, 'Instagram Comment Chi Fitness']],columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])

            df = pd.concat([df2,df], ignore_index=True)

            # sum greater than 0 => positive sentiment
            if sentiment >= 0:
                print("This sentence has a positive sentiment: " + str(sentiment))
            elif sentiment < 0:
                print("This sentence has a negative sentiment: " + str(sentiment))
        print()
    except:
        continue

New comment
---------------------------------------
Sentence:  hye, i’ve sent you a message.
[('hye', 'NN'), (',', ','), ('’', 'NNP'), ('ve', 'FW'), ('sent', 'NN'), ('you', 'PRP'), ('a', 'DT'), ('message', 'NN'), ('.', '.')]
Lemmatized word:  hye
Synset not found
Lemmatized word:  ,
Synset not found
Lemmatized word:  ’
Synset not found
Lemmatized word:  ve
Synset not found
Lemmatized word:  sent
sent  |  Synset('sent.n.01')  |  <sent.n.01: PosScore=0.0 NegScore=0.0>  |  100 senti equal 1 kroon in Estonia
Lemmatized word:  you
Synset not found
Lemmatized word:  a
a  |  Synset('angstrom.n.01')  |  <angstrom.n.01: PosScore=0.0 NegScore=0.0>  |  a metric unit of length equal to one ten billionth of a meter (or 0.0001 micron); used to specify wavelengths of electromagnetic radiation
Lemmatized word:  message
message  |  Synset('message.n.01')  |  <message.n.01: PosScore=0.0 NegScore=0.0>  |  a communication (usually brief) that is written or spoken or signaled
Lemmatized word:  .
Synset not

[('it', 'PRP'), ("'ll", 'MD'), ('be', 'VB'), ('great', 'JJ'), ('to', 'TO'), ('get', 'VB'), ('a', 'DT'), ('cordless', 'NN'), ('earphone', 'NN'), ('to', 'TO'), ('replace', 'VB'), ('my', 'PRP$'), ('current', 'JJ'), ('earpods', 'NNS'), ('to', 'TO'), ('be', 'VB'), ('used', 'VBN'), ('during', 'IN'), ('work', 'NN'), ('out', 'IN'), (':', ':'), (')', ')')]
Lemmatized word:  it
it  |  Synset('information_technology.n.01')  |  <information_technology.n.01: PosScore=0.0 NegScore=0.0>  |  the branch of engineering that deals with the use of computers and telecommunications to retrieve and store and transmit information
Lemmatized word:  'll
Synset not found
Lemmatized word:  be
be  |  Synset('be.v.01')  |  <be.v.01: PosScore=0.25 NegScore=0.125>  |  have the quality of being; (copula, used with an adjective or a predicate noun)
Lemmatized word:  great
great  |  Synset('great.s.01')  |  <great.s.01: PosScore=0.0 NegScore=0.0>  |  relatively large in size or number or extent; larger than others of it

it  |  Synset('information_technology.n.01')  |  <information_technology.n.01: PosScore=0.0 NegScore=0.0>  |  the branch of engineering that deals with the use of computers and telecommunications to retrieve and store and transmit information
Lemmatized word:  ’
Synset not found
Lemmatized word:  ll
Synset not found
Lemmatized word:  be
be  |  Synset('be.v.01')  |  <be.v.01: PosScore=0.25 NegScore=0.125>  |  have the quality of being; (copula, used with an adjective or a predicate noun)
Lemmatized word:  awesome
Synset not found
Lemmatized word:  to
Synset not found
Lemmatized word:  get
get  |  Synset('get.v.01')  |  <get.v.01: PosScore=0.125 NegScore=0.0>  |  come into the possession of something concrete or abstract
Lemmatized word:  wireless
wireless  |  Synset('wireless.a.01')  |  <wireless.a.01: PosScore=0.0 NegScore=0.0>  |  having no wires
Lemmatized word:  earphone
earphone  |  Synset('earphone.n.01')  |  <earphone.n.01: PosScore=0.0 NegScore=0.0>  |  electro-acoustic transduc

Sentence:  🙋🏻🙋🏻🙋🏻
[('🙋🏻🙋🏻🙋🏻', 'NN')]
Lemmatized word:  🙋🏻🙋🏻🙋🏻
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  🙋🏼‍♀️
[('🙋🏼\u200d♀️', 'NN')]
Lemmatized word:  🙋🏼‍♀️
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  🙋🏻‍♀️
[('🙋🏻\u200d♀️', 'NN')]
Lemmatized word:  🙋🏻‍♀️
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  🙋🏻‍♀️
[('🙋🏻\u200d♀️', 'NN')]
Lemmatized word:  🙋🏻‍♀️
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  ♂
[('♂', 'NN')]
Lemmatized word:  ♂
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  🙋 ♂
[('🙋', 'NN'), ('♂', 'NN')]
Lemmatized word:  🙋
Synset not found
Lemmatized word:  ♂
Synset not found
This sentence

[('(', '('), (':', ':'), ('btw', 'NN'), (',', ','), ('already', 'RB'), ('.', '.')]
Lemmatized word:  (
Synset not found
Lemmatized word:  :
Synset not found
Lemmatized word:  btw
Synset not found
Lemmatized word:  ,
Synset not found
Lemmatized word:  already
already  |  Synset('already.r.01')  |  <already.r.01: PosScore=0.125 NegScore=0.0>  |  prior to a specified or implied time
Lemmatized word:  .
Synset not found
This sentence has a positive sentiment: 0.125

New comment
---------------------------------------
Sentence:  @  i alreafy follow 😇
[('@', 'NN'), ('alreafy', 'NN'), ('follow', 'VBP'), ('😇', 'NN')]
Lemmatized word:  @
Synset not found
Lemmatized word:  alreafy
Synset not found
Lemmatized word:  follow
follow  |  Synset('follow.v.01')  |  <follow.v.01: PosScore=0.0 NegScore=0.0>  |  to travel behind, go after, come after
Lemmatized word:  😇
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  thank you @ 

...

Sentence:  @felicia.fang you show this face to @daphling  hahahaha
[('@', 'NN'), ('felicia.fang', 'NN'), ('you', 'PRP'), ('show', 'VBP'), ('this', 'DT'), ('face', 'NN'), ('to', 'TO'), ('@', 'VB'), ('daphling', 'VBG'), ('hahahaha', 'NN')]
Lemmatized word:  @
Synset not found
Lemmatized word:  felicia.fang
Synset not found
Lemmatized word:  you
Synset not found
Lemmatized word:  show
show  |  Synset('show.v.01')  |  <show.v.01: PosScore=0.0 NegScore=0.0>  |  give an exhibition of to an interested audience
Lemmatized word:  this
Synset not found
Lemmatized word:  face
face  |  Synset('face.n.01')  |  <face.n.01: PosScore=0.0 NegScore=0.0>  |  the front of the human head from the forehead to the chin and ear to ear
Lemmatized word:  to
Synset not found
Lemmatized word:  @
Synset not found
Lemmatized word:  daphling
Synset not found
Lemmatized word:  hahahaha
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  awesome


Sentence:  nice shoes
[('nice', 'NN'), ('shoes', 'NNS')]
Lemmatized word:  nice
nice  has a score of 0.875
Lemmatized word:  shoe
shoe  |  Synset('shoe.n.01')  |  <shoe.n.01: PosScore=0.0 NegScore=0.0>  |  footwear shaped to fit the foot (below the ankle) with a flexible upper of leather or plastic and a sole and heel of heavier material
This sentence has a positive sentiment: 0.875

New comment
---------------------------------------
Sentence:  hi, i got urgent enquiry need ask.
[('hi', 'NN'), (',', ','), ('got', 'VBD'), ('urgent', 'JJ'), ('enquiry', 'NNS'), ('need', 'VBP'), ('ask', 'NNS'), ('.', '.')]
Lemmatized word:  hi
hi  |  Synset('hello.n.01')  |  <hello.n.01: PosScore=0.0 NegScore=0.0>  |  an expression of greeting
Lemmatized word:  ,
Synset not found
Lemmatized word:  get
get  |  Synset('get.v.01')  |  <get.v.01: PosScore=0.125 NegScore=0.0>  |  come into the possession of something concrete or abstract
Lemmatized word:  urgent
urgent  |  Synset('pressing.s.01')  |  <pressing

Sentence:  @xzikri cuz pizza is the diet.
[('@', 'JJ'), ('xzikri', 'NNP'), ('cuz', 'NN'), ('pizza', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('diet', 'NN'), ('.', '.')]
Lemmatized word:  @
Synset not found
Lemmatized word:  xzikri
Synset not found
Lemmatized word:  cuz
Synset not found
Lemmatized word:  pizza
pizza  |  Synset('pizza.n.01')  |  <pizza.n.01: PosScore=0.0 NegScore=0.0>  |  Italian open pie made of thin bread dough spread with a spiced mixture of e.g. tomato sauce and cheese
Lemmatized word:  be
be  |  Synset('be.v.01')  |  <be.v.01: PosScore=0.25 NegScore=0.125>  |  have the quality of being; (copula, used with an adjective or a predicate noun)
Lemmatized word:  the
Synset not found
Lemmatized word:  diet
diet  |  Synset('diet.n.01')  |  <diet.n.01: PosScore=0.0 NegScore=0.125>  |  a prescribed selection of foods
Lemmatized word:  .
Synset not found
This sentence has a positive sentiment: 0.0
Sentence:  hahahahah
[('hahahahah', 'NN')]
Lemmatized word:  hahahahah
Synset not fo

Sentence:  @jaydonpang @ kabir.a.sultan
[('@', 'NN'), ('jaydonpang', 'NN'), ('@', 'NNP'), ('kabir.a.sultan', 'NN')]
Lemmatized word:  @
Synset not found
Lemmatized word:  jaydonpang
Synset not found
Lemmatized word:  @
Synset not found
Lemmatized word:  kabir.a.sultan
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  @holiday668
[('@', 'NN'), ('holiday668', 'NN')]
Lemmatized word:  @
Synset not found
Lemmatized word:  holiday668
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  😎
[('😎', 'NN')]
Lemmatized word:  😎
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  💪👍⚽
[('💪👍⚽', 'NN')]
Lemmatized word:  💪👍⚽
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  💥🔝👆
[('💥🔝👆', 'NN')]
Lemmatized word:  💥

Sentence:  impressive!
[('impressive', 'JJ'), ('!', '.')]
Lemmatized word:  impressive
impressive  |  Synset('impressive.a.01')  |  <impressive.a.01: PosScore=0.125 NegScore=0.0>  |  making a strong or vivid impression
Lemmatized word:  !
Synset not found
This sentence has a positive sentiment: 0.125
Sentence:  👍
[('👍', 'NN')]
Lemmatized word:  👍
Synset not found
This sentence has a positive sentiment: 0.0

New comment
---------------------------------------
Sentence:  so healthy
[('so', 'RB'), ('healthy', 'JJ')]
Lemmatized word:  so
so  |  Synset('so.r.01')  |  <so.r.01: PosScore=0.0 NegScore=0.0>  |  to a very great extent or degree
Lemmatized word:  healthy
healthy  |  Synset('healthy.a.01')  |  <healthy.a.01: PosScore=0.75 NegScore=0.0>  |  having or indicating good health in body or mind; free from infirmity or disease
This sentence has a positive sentiment: 0.75

New comment
---------------------------------------
Sentence:  very nice  gallery!
[('very', 'RB'), ('nice', 'JJ'), ('

In [6]:
df_filtered = df[df['Sentiment'] != 0]
print(len(df_filtered[df_filtered['Sentiment'] < 0]))
print(len(df_filtered[df_filtered['Sentiment'] > 0]))

94
264


In [7]:
df_filtered[df_filtered['Sentiment'] > 0]

,Post,Comment,Topic,Sentiment,Source
1,Not tomorrow. Right now. #transformation #weig...,very nice gallery!,gallery,0.875,Instagram Comment Chi Fitness
2,It's Wednesday. Go work out. #eatclean #trainmean,so healthy,,0.750,Instagram Comment Chi Fitness
4,#BeBetterThanBetter #fitfam #fitspo #malaysia ...,impressive!,,0.125,Instagram Comment Chi Fitness
8,Happy faces for an early weekend! 😄\nWho's goi...,really ?,,0.625,Instagram Comment Chi Fitness
10,Happy faces for an early weekend! 😄\nWho's goi...,i don't have workout partner but do you have o...,"workout,partner",0.500,Instagram Comment Chi Fitness
12,Happy faces for an early weekend! 😄\nWho's goi...,we love it!,,0.500,Instagram Comment Chi Fitness
13,Happy faces for an early weekend! 😄\nWho's goi...,nice picture,picture,0.875,Instagram Comment Chi Fitness
14,Flipping out after this 5 day weekend with @mu...,"hi , are you iphone user?if ade issue, just l...","issue,k",0.250,Instagram Comment Chi Fitness
16,Flipping out after this 5 day weekend with @mu...,nice one @muhd3mustaqim!!,,0.875,Instagram Comment Chi Fitness
33,Breaking it down with the legend @soudyfitness...,jom usha ig kami 😊😄😚,"ig,kami",0.125,Instagram Comment Chi Fitness


In [8]:
df_filtered[df_filtered['Sentiment'] < 0]

,Post,Comment,Topic,Sentiment,Source
93,Yooooooo! We're dropping deals early..Don't fo...,fake promotion.,promotion,-0.750,Instagram Comment Chi Fitness
102,Congratulations to @aliciaaaleow and @alvinooi...,@fabianng90 🤦🏻‍♂️🤦🏻‍♂️🤦🏻‍♂️ fabian we can no n...,"fabian,workout",-0.250,Instagram Comment Chi Fitness
112,Pop quiz! What does RPM stand for? \nA) Rate P...,"sorry dont know, dont do cardio m8",,-0.625,Instagram Comment Chi Fitness
132,"…and IT BEGINS NOW! \nNaughty or nice, just fo...",i sweat when i start getting into my deadlift ...,"sweat,start,stance,form,sucks,people,time,chri...",-1.375,Instagram Comment Chi Fitness
138,"…and IT BEGINS NOW! \nNaughty or nice, just fo...",no more excuses!,excuses,-0.500,Instagram Comment Chi Fitness
139,"…and IT BEGINS NOW! \nNaughty or nice, just fo...",i sweat when i think of all the work i haven't...,"sweat,think,work,procrastination,habit",-0.375,Instagram Comment Chi Fitness
141,"…and IT BEGINS NOW! \nNaughty or nice, just fo...",i sweat when i do insanity!!!,"sweat,insanity",-0.125,Instagram Comment Chi Fitness
143,"…and IT BEGINS NOW! \nNaughty or nice, just fo...",i sweat when i do insanity!!,"sweat,insanity",-0.125,Instagram Comment Chi Fitness
144,"…and IT BEGINS NOW! \nNaughty or nice, just fo...",[sing them by the tune of baa baa black sheep]...,"tune,baa,baa,sheep",-0.375,Instagram Comment Chi Fitness
151,"…and IT BEGINS NOW! \nNaughty or nice, just fo...",i stop making myself feeling upset because i k...,"don,t",-1.500,Instagram Comment Chi Fitness


In [9]:
df_filtered.to_json('instagram_chifitnesssentiment.json', orient='records')